<a href="https://colab.research.google.com/github/hm06063/hihi/blob/master/DQN1(Sung_Kim).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb \
    xorg-dev python-opengl libboost-all-dev libsdl2-dev swig
!pip install pyvirtualdisplay
!pip install piglet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
python-dev is already the newest version (2.7.15~rc1-1).
python-numpy is already the newest version (1:1.13.3-2ubuntu1).
python-numpy set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libboost-all-dev is already the newest version (1.65.1.0ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
The following additional packages will be installed:
  gir1.2-ibus-1.0 libcapnp-0.6.1 libdbus-1-dev libdmx-dev libdmx1
  libfontenc-dev libfs-dev libfs6 libibus-1.0-5 libibus-1.0-dev
  libmirclient-dev libmirclient9 libmircommon-dev libmircommon7
  libmircookie-dev libmircookie2 libmircore-dev libmircore1 libmirprotobuf3
  libpciaccess-dev libpixman-1-dev libprotobuf-dev libprotobuf-lite10
  libpulse-dev libpu

In [0]:
!apt-get install x11-utils
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
def show_video():
    mp4_list = glob('video/*.mp4')
    if mp4_list:
        mp4 = mp4_list[0]
        video = open(mp4, 'r+b').read()
        encoded = b64encode(video)
        ipy_display.display(HTML(data='''
            <video alt="gameplay" autoplay controls style="height: 400px;">
                <source src="data:video/mp4;base64,%s" type="video/mp4" />
            </video>
        ''' % (encoded.decode('ascii'))))
    else:
        print('No video found')

def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

1. Network Build & Initalize
2. Environment Build & Initalize
3. loop를 돌며 action을 가져와 (epsilon 사용)
4. action을 취해서 r이나 s가 돌아오면 이걸 가지고
5. 학습시키지 않고 버퍼에 저장해.
6. 저장시키는 루프를 돌다가 어느 순간 random sample해서 network을 학습시켜.

In [0]:
import numpy as np
import tensorflow as tf
import random
import dqn####################다른 클래스 파일 코랩에서 import 어케하는지 몰라서 jupyter로 넘어감,ㅠㅠ
from collections import deque

import gym
env = gym.make('CartPole-v0')

#신경망 정의하는 상수들
input_size = env.observation_space.shape[0] #네개의 정보
output_size = env.action_space.n #두개의 action

dis = 0.9
REPLAY_MEMORY = 50000

In [0]:
class DQN:

  #Session 받아오기 중요!
  def __init__(self, session, input_size, output_size, name = "main"):
    self.session = session
    self.input_size = input_size
    self.output_size = output_size
    self.net_name = name

    self._build_network()

  #Network Build
  def _build_network(self, h_size=10, l_rate = 1e-1):
    with tf.variable_scope(self.net_name):
      self._X = tf.placeholder(
          tf.float32, [None,self.input_size], name = "input_x") 
      #입력사이즈 만큼 받기. None - 입력 한개~여러개 동시에 받기 위해서
      
      #First layer of Weights
      W1 = tf.get_variable("W1", shape=[self.input_size, h_size], 
                           initializer=tf.contrib.layers.xavier_initializer())
      layer1 = tf.nn.tanh(tf.matmul(self._X,W1))
      #tanh : activation function

      #Second layer of Weights
      W2 = tf.get_variable("W2", shape=[h_size, self.output_size], #원하는 outputsize
                           initializer=tf.contrib.layers.xavier_initializer())
      
      #마지막 layer는 activation function해주지 않아도 됨. Q가 linear regression이라서

      #Q prediction
      self._Qpred = tf.matmul(layer1, W2)

    #We need to define the parts of the network needed for learning a policy
    self._Y = tf.placeholder(
        shape=[None,self.output_size],dtype=tf.float32)
    #Y는 정답. Y를 hold할 수 있는 공간 생성!
  
    
    #Loss function
    self._loss = tf.reduce_mean(tf.square(self._Y - self._Qpred))
    #Learning 학습시킬 때마다 train호출!
    self._train = tf.train.AdamOptimizer(
        learning_rate = l_rate).minimize(self._loss)
    
  #상태를 받아서 실행시킨 다음에 결과를 받아주는 함수
  def predict(self,state):
    x = np.reshape(state,[1,self.input_size])
    return self.session.run(self._Qpred, feed_dict={self._X: x})

  #학습시키는 함수. 
  def update(self,x_stack, y_stack):
    return self.session.run([self._loss, self._train],feed_dict={
        self._X: x_stack, self._Y: y_stack})

2.Replay Memory

 *원하는 사이즈를 지정해두고 그 사이즈를 넘기면 pop해서 사이즈를 유지시켜줌.

 *epsiode가 몇번될때마다 한 번씩 학습시켜줄지 정해줌. 그 때마다 몇 번정도 돌면서 replay buffer에서 몇개씩 랜덤하게 뽑아와 학습시킴.

 Replay memory에 있는 값을 어떻게 학습시킬까?
 한개씩 학습시키지말고 모아서 학습시키자는 의미로 stack을 사용. (vstack : vertical stack -> np.vstack([A,B])

빈 stack을 생성한 후, 미니배치에서 state, action 등을 하나씩 가져옴. 그리고 state값을 predict 함수에 넣어서 Q를 가져와. 그리고 Q에 대한 action을 터미널이냐 아니냐에 따라 나눠서 업데이트 시켜줘.
*cf) DQN.predict(next_state) : Q-value

이 때 Q값들을 y_stack에 쌓고, state값을 x_stack에 쌓아서 한번에 update시켜!

In [0]:
#Train from Replay Buffer ## update는 메인, 읽어오는 것은 target!!
def replay_train(mainDQN, targetDQN, train_batch):
  x_stack = np.empty(0).reshape(0,input_size)
  y_stack = np.empty(0).reshape(0,output_size)

  #Get stored info from the buffer
  for state, action, reward, next_state, done in train_batch:
    Q = mainDQN.predict(state)

    #Terminal?
    if done:
      Q[0,action] = reward
    else:
      #Q-value를 가져와(by feeding the new_state through our network)
      Q[0,action] = reward + dis * np.max(targetDQN.predict(next_state))

    y_stack = np.vstack([y_stack,Q])
    x_stack = np.vstack([x_stack,state])

    #Train our network using target and predicted Q values on each epsiode
    return mainDQN.update(x_stack, y_stack)

In [0]:
#Code 5 : network(variable) copy
def get_copy_var_ops(*, dest_scope_name = "target", src_scope_name = "main"):
  #Copy variables src_scope to dest_scope
  op_holder =[]

  src_vars = tf.get_collection(
      tf.GraphKeys.TRAINABLE_VARIABLES, scope = src_scope_name)
  dest_vars = f.get_collection(
      tf.GraphKeys.TRAINABLE_VARIABLES, scope = dest_scope_name)
  
  for src_var, dest_var in zip(src_vars,dest_vars):
    op_holder.append(dest_var.assign(src_var.value()))

  return op_holder
  

In [0]:
#bot play - 학습된 network 받아와서 환경 초기화후 계산 결과 보여줌
def bot_play(mainDQN):
  #See our trained network in action
  s = env.reset()
  reward_sum = 0
  while True:
    env.render()
    a = np.argmax(mainDQN.predict(s))
    s, reward, done, _ = env.step(a)
    reward_sum += reward
    if done:
      print("Total score :{}".format(reward_sum))
      break

In [0]:
#main
def main():
  max_episodes = 5000

  #state the previous observations in replay memory
  replay_buffer = deque()

  #네트워크 생성
  with tf.Session() as sess:
    mainDQN = dqn.DQN(sess, input_size, output_size, name = "main")
    targetDQN = dqn.DQN(sess, input_size, output_size, name = "target")
    tf.global_variables_initializer().run()

     #initial copy q_net -> target_net
    copy_ops = get_copy_var_ops(dest_scope_name = "target", 
                                src_scope_name = "main")
    sess.run(copy_ops)

    for episodes in range(max_episodes):
      e = 1. / ((epsode/10)+1)
      done = False
      step_count = 0
      state = env.reset()

      while not done:
        if np.random.rand(1) <e:
          action = env.action_space.sample()
        else:
          #Choose an action by greedily from the Q Network
          action = np.argmax(mainDQN.predict(state))

        #Get new state and reward from environment
        next_state, reward, done, _ = env.step(action)
        if done: #big penalty
          reward = -100
        
        #save the 경험 to our buffer
        replay_buffer.append((state, action, reward, next_state, done))
        if len(replay_buffer)>REPLAY_MEMORY:
          replay_buffer.popleft()

        state = next_state
        step_count += 1
        if step_count > 10000: #충분
          break

        print("Episodes: {}, steps:{}".format(episode, step_count))
        if step_count>10000:
          pass #break
        
        if episode % 10 == 1: #train every 10 episodes
        #Get a random batch of experience
          for _ in range(50):
            #Minibatch works better
            minibatch = random.sample(replay_buffer,10)
            loss, _ = simple_replay_train(mainDQN, minibatch)
          print("Loss : ", loss)

          #copy q_net -> target_net
          sess.run(copy_ops)

        bot_play(mainDQN)
    if __name__ == "__main__":
      main()


In [0]:
#See our trained bot in action
from gym import wrappers
env2 = wrappers.Monitor(env,'gym-results',force= True)

for i in range(200):
  bot_play(mainDQN,env=env2)

NameError: ignored